In [ ]:
# | default_exp prepare_soil_parameters

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import os
import warnings
import collections
import numpy as np
from pathlib import Path
from typing import Dict, List
from dataclasses import dataclass, field
from pysureau.soil_utils import (
    read_soil_file,
    compute_theta_at_given_p_soil,
    compute_theta_at_given_p_soil_camp,
)
from pysureau.conversions_utils import convert_vwc_to_sws, convert_f_to_v
# from sureau_ecos_py.create_modeling_options import create_modeling_options

In [ ]:
def prepare_soil_parameters(
    file_path: Path = None,  # Path to the file created with the `soil_parameter_file` function
) -> Dict:  # Dictionary containing soil parameters parameters
    "Create a Dictionary with soil parameters to run SureauR"

    # Read soil parameter CSV file ----------------------------------------------

    soil_parameters_dict = read_soil_file(file_path, sep=',')

    # Reshape parameters --------------------------------------------------------

    # Create single array with Layer thickness
    soil_parameters_dict['soil_layers_thickness'] = collections.defaultdict(
        list,
        {
            'layer_1': soil_parameters_dict['soil_depth_1'],
            'layer_2': soil_parameters_dict['soil_depth_2']
            - soil_parameters_dict['soil_depth_1'],
            'layer_3': soil_parameters_dict['soil_depth_3']
            - soil_parameters_dict['soil_depth_2'],
        },
    )

    soil_parameters_dict['rfc'] = collections.defaultdict(
        list,
        {
            'layer_1': soil_parameters_dict['rfc_1'],
            'layer_2': soil_parameters_dict['rfc_2'],
            'layer_3': soil_parameters_dict['rfc_3'],
        },
    )
    
    # Remove kye-value pairs
    soil_parameters_dict.pop('rfc_1', None)
    soil_parameters_dict.pop('rfc_2', None)
    soil_parameters_dict.pop('rfc_3', None)
    
    
    soil_parameters_dict['soil_depth'] = collections.defaultdict(
        list,
        {
            'layer_1': soil_parameters_dict['soil_depth_1'],
            'layer_2': soil_parameters_dict['soil_depth_2'],
            'layer_3': soil_parameters_dict['soil_depth_3'],
        },
    )
    
    # Remove kye-value pairs
    soil_parameters_dict.pop('soil_depth_1', None)
    soil_parameters_dict.pop('soil_depth_2', None)
    soil_parameters_dict.pop('soil_depth_3', None)

    # Add new soil params for van genuchten formulation -------------------------
    
    if soil_parameters_dict['pedo_transfer_formulation'] == 'vg':
        # Shape parameters of the relationship between soil water content and
        # soil water potential

        # Shape parameter 1
        soil_parameters_dict['alpha_vg'] = collections.defaultdict(
            list,
            {
                'layer_1': soil_parameters_dict['alpha_vg'],
                'layer_2': soil_parameters_dict['alpha_vg'],
                'layer_3': soil_parameters_dict['alpha_vg'],
            },
        )

        # Shape parameter 2

        soil_parameters_dict['n_vg'] = collections.defaultdict(
            list,
            {
                'layer_1': soil_parameters_dict['n_vg'],
                'layer_2': soil_parameters_dict['n_vg'],
                'layer_3': soil_parameters_dict['n_vg'],
            },
        )

        # Shape parameter 3

        soil_parameters_dict['i_vg'] = collections.defaultdict(
            list,
            {
                'layer_1': soil_parameters_dict['i_vg'],
                'layer_2': soil_parameters_dict['i_vg'],
                'layer_3': soil_parameters_dict['i_vg'],
            },
        )

        # Soil conductivity at saturation (mol/m/s/Mpa)
        soil_parameters_dict['ksat_vg'] = collections.defaultdict(
            list,
            {
                'layer_1': soil_parameters_dict['ksat_vg'],
                'layer_2': soil_parameters_dict['ksat_vg'],
                'layer_3': soil_parameters_dict['ksat_vg'],
            },
        )
        # Fraction of water at saturation capacity (cm3/cm3)
        soil_parameters_dict['saturation_capacity_vg'] = collections.defaultdict(
            list,
            {
                'layer_1': soil_parameters_dict['saturation_capacity_vg'],
                'layer_2': soil_parameters_dict['saturation_capacity_vg'],
                'layer_3': soil_parameters_dict['saturation_capacity_vg'],
            },
        )

        # Fraction of residual water (cm3/cm3)
        soil_parameters_dict['residual_capacity_vg'] = collections.defaultdict(
            list,
            {
                'layer_1': soil_parameters_dict['residual_capacity_vg'],
                'layer_2': soil_parameters_dict['residual_capacity_vg'],
                'layer_3': soil_parameters_dict['residual_capacity_vg'],
            },
        )

        # Add computation of wilting point
        soil_parameters_dict['wilting_point'] = collections.defaultdict(
            list,
            {
                'layer_1': compute_theta_at_given_p_soil(
                    psi_target=1.5,
                    theta_res=soil_parameters_dict['residual_capacity_vg'][
                        'layer_1'
                    ],
                    theta_sat=soil_parameters_dict['saturation_capacity_vg'][
                        'layer_1'
                    ],
                    alpha_vg=soil_parameters_dict['alpha_vg']['layer_1'],
                    n_vg=soil_parameters_dict['n_vg']['layer_1'],
                ),
                'layer_2': compute_theta_at_given_p_soil(
                    psi_target=1.5,
                    theta_res=soil_parameters_dict['residual_capacity_vg'][
                        'layer_2'
                    ],
                    theta_sat=soil_parameters_dict['saturation_capacity_vg'][
                        'layer_2'
                    ],
                    alpha_vg=soil_parameters_dict['alpha_vg']['layer_2'],
                    n_vg=soil_parameters_dict['n_vg']['layer_2'],
                ),
                'layer_3': compute_theta_at_given_p_soil(
                    psi_target=1.5,
                    theta_res=soil_parameters_dict['residual_capacity_vg'][
                        'layer_2'
                    ],
                    theta_sat=soil_parameters_dict['saturation_capacity_vg'][
                        'layer_2'
                    ],
                    alpha_vg=soil_parameters_dict['alpha_vg']['layer_2'],
                    n_vg=soil_parameters_dict['n_vg']['layer_2'],
                ),
            },
        )

        # Add computation of field capacity from functions
        soil_parameters_dict['field_capacity'] = collections.defaultdict(
            list,
            {
                'layer_1': compute_theta_at_given_p_soil(
                    psi_target=soil_parameters_dict['psoil_at_field_capacity'],
                    theta_res=soil_parameters_dict['residual_capacity_vg'][
                        'layer_1'
                    ],
                    theta_sat=soil_parameters_dict['saturation_capacity_vg'][
                        'layer_1'
                    ],
                    alpha_vg=soil_parameters_dict['alpha_vg']['layer_1'],
                    n_vg=soil_parameters_dict['n_vg']['layer_1'],
                ),
                'layer_2': compute_theta_at_given_p_soil(
                    psi_target=soil_parameters_dict['psoil_at_field_capacity'],
                    theta_res=soil_parameters_dict['residual_capacity_vg'][
                        'layer_2'
                    ],
                    theta_sat=soil_parameters_dict['saturation_capacity_vg'][
                        'layer_2'
                    ],
                    alpha_vg=soil_parameters_dict['alpha_vg']['layer_2'],
                    n_vg=soil_parameters_dict['n_vg']['layer_2'],
                ),
                'layer_3': compute_theta_at_given_p_soil(
                    psi_target=soil_parameters_dict['psoil_at_field_capacity'],
                    theta_res=soil_parameters_dict['residual_capacity_vg'][
                        'layer_3'
                    ],
                    theta_sat=soil_parameters_dict['saturation_capacity_vg'][
                        'layer_3'
                    ],
                    alpha_vg=soil_parameters_dict['alpha_vg']['layer_3'],
                    n_vg=soil_parameters_dict['n_vg']['layer_3'],
                ),
            },
        )
        
        # v_ parameters ---------------------------------------------------------
        soil_parameters_dict['v_field_capacity_vg'] = (
                collections.defaultdict(
                    list,
                    {'layer_1':convert_f_to_v(x=soil_parameters_dict['field_capacity']['layer_1'],
                                                     rfc=soil_parameters_dict['rfc']['layer_1'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                     
                    'layer_2':convert_f_to_v(x=soil_parameters_dict['field_capacity']['layer_2'],
                                                     rfc=soil_parameters_dict['rfc']['layer_2'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                    
                    'layer_3':convert_f_to_v(x=soil_parameters_dict['field_capacity']['layer_3'],
                                                     rfc=soil_parameters_dict['rfc']['layer_3'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],),
                    }
                )
            )
        
        soil_parameters_dict['v_field_capacity_vg'] = (
                collections.defaultdict(
                    list,
                    {'layer_1':convert_f_to_v(x=soil_parameters_dict['saturation_capacity_vg']['layer_1'],
                                                     rfc=soil_parameters_dict['rfc']['layer_1'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                     
                    'layer_2':convert_f_to_v(x=soil_parameters_dict['saturation_capacity_vg']['layer_2'],
                                                     rfc=soil_parameters_dict['rfc']['layer_2'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                    
                    'layer_3':convert_f_to_v(x=soil_parameters_dict['saturation_capacity_vg']['layer_3'],
                                                     rfc=soil_parameters_dict['rfc']['layer_3'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],),
                    }
                )
            )
    
        soil_parameters_dict['v_residual_capacity_vg'] = (
                collections.defaultdict(
                    list,
                    {'layer_1':convert_f_to_v(x=soil_parameters_dict['residual_capacity_vg']['layer_1'],
                                                     rfc=soil_parameters_dict['rfc']['layer_1'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                     
                    'layer_2':convert_f_to_v(x=soil_parameters_dict['residual_capacity_vg']['layer_2'],
                                                     rfc=soil_parameters_dict['rfc']['layer_2'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                    
                    'layer_3':convert_f_to_v(x=soil_parameters_dict['residual_capacity_vg']['layer_3'],
                                                     rfc=soil_parameters_dict['rfc']['layer_3'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],),
                    }
                )
            )

    
    
        # Add v_wilting_point
        soil_parameters_dict['v_wilting_point'] = (
                collections.defaultdict(
                    list,
                    {'layer_1':convert_f_to_v(x=soil_parameters_dict['wilting_point']['layer_1'],
                                                     rfc=soil_parameters_dict['rfc']['layer_1'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                     
                    'layer_2':convert_f_to_v(x=soil_parameters_dict['wilting_point']['layer_2'],
                                                     rfc=soil_parameters_dict['rfc']['layer_2'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                    
                    'layer_3':convert_f_to_v(x=soil_parameters_dict['wilting_point']['layer_3'],
                                                     rfc=soil_parameters_dict['rfc']['layer_3'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],),
                    }
                )
            )
    #            # For diagnostic, Unknown reason why
    #            soil_parameters_dict['v_soil_storage_capacity_wilt'] = np.sum(soil_parameters_dict['v_field_capacity']) - np.sum(soil_parameters_dict['v_wilting_point'])
    #            soil_parameters_dict['v_soil_storage_capacity_res'] = np.sum(soil_parameters_dict['v_field_capacity']) - np.sum(soil_parameters_dict['v_residual_capacity_vg'])
    #            soil_parameters_dict['v_soil_storage_capacity'] = soil_parameters_dict['v_soil_storage_capacity_wilt']
    #
    #            print(
    #                f'Available water capacity Wilting: {soil_parameters_dict["v_soil_storage_capacity_wilt"]} mm'
    #            )
    #            print(
    #                f'Available water capacity Residual: {soil_parameters_dict["v_soil_storage_capacity_res"]} mm'
    #            )
    #
    #            print('Can soil_params["v_soil_storage_capacity"] be negative?? Ask')
    #
    
    # Add new soil params for campbell formulation ------------------------------
    elif soil_parameters_dict['pedo_transfer_formulation'] == "campbell":
        
    
                # Shape parameters of the relationship between soil water content and
                # soil water potential
    
                soil_parameters_dict['b_camp'] = collections.defaultdict(list,{
                        'layer_1': soil_parameters_dict['b_camp'],
                        'layer_2': soil_parameters_dict['b_camp'],
                        'layer_3': soil_parameters_dict['b_camp'],},)

    
                # Shape parameter 2
                soil_parameters_dict['psie'] = collections.defaultdict(list,{
                        'layer_1': soil_parameters_dict['psie'],
                        'layer_2': soil_parameters_dict['psie'],
                        'layer_3': soil_parameters_dict['psie'],},)

                # Soil conductivity at saturation (mol/m/s/Mpa)
                # Value not repeated 3 times as the ksat_vg param
                soil_parameters_dict['ksat_campbell'] = soil_parameters_dict['ksat_campbell']
    
    
                # Fraction of water at saturation capacity (cm3/cm3)                
                soil_parameters_dict['saturation_capacity_campbell'] = collections.defaultdict(list,{
                        'layer_1': soil_parameters_dict['saturation_capacity_campbell'],
                        'layer_2': soil_parameters_dict['saturation_capacity_campbell'],
                        'layer_3': soil_parameters_dict['saturation_capacity_campbell'],},)
                    
                # Add wilting point ---------------------------------------------
        
                soil_parameters_dict['wilting_point'] = collections.defaultdict(
                    list,{
                        'layer_1': compute_theta_at_given_p_soil_camp(psi_target=-1.5,
                                                                      theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_1'],
                                                                      psie=soil_parameters_dict['psie']['layer_1'],
                                                                      b_camp=soil_parameters_dict['b_camp']['layer_1'],),
                          
                          'layer_2': compute_theta_at_given_p_soil_camp(psi_target=-1.5,
                                                                        theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_2'],
                                                                        psie=soil_parameters_dict['psie']['layer_2'],
                                                                        b_camp=soil_parameters_dict['b_camp']['layer_2'],),
                          
                          'layer_3': compute_theta_at_given_p_soil_camp(psi_target=-1.5,
                                                                        theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_3'],
                                                                        psie=soil_parameters_dict['psie']['layer_3'],
                                                                        b_camp=soil_parameters_dict['b_camp']['layer_3'],)},)
    
                # Add field capacity --------------------------------------------
                soil_parameters_dict['field_capacity'] = collections.defaultdict(
                    list,{
                        'layer_1': compute_theta_at_given_p_soil_camp(psi_target=soil_parameters_dict['psoil_at_field_capacity'],
                                                              theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_1'],
                                                              psie=soil_parameters_dict['psie']['layer_1'],
                                                              b_camp=soil_parameters_dict['b_camp']['layer_1']),
                
                        'layer_2': compute_theta_at_given_p_soil_camp(psi_target=soil_parameters_dict['psoil_at_field_capacity'],
                                           theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_2'],
                                           psie=soil_parameters_dict['psie']['layer_2'],
                                           b_camp=soil_parameters_dict['b_camp']['layer_2']),
                                   
                        'layer_3':compute_theta_at_given_p_soil_camp(psi_target=soil_parameters_dict['psoil_at_field_capacity'],
                                           theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_3'],
                                           psie=soil_parameters_dict['psie']['layer_3'],
                                           b_camp=soil_parameters_dict['b_camp']['layer_3'])})
                #
                # Add residual capacity camp,Fraction of residual water ---------
                # (cm3/cm3)
                soil_parameters_dict['residual_capacity_campbell'] = collections.defaultdict(
                    list,{'layer_1': compute_theta_at_given_p_soil_camp(psi_target=-100,
                                                                        theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_1'],
                                                                        psie=soil_parameters_dict['psie']['layer_1'],
                                                                        b_camp=soil_parameters_dict['b_camp']['layer_1'],),
                          
                          'layer_2': compute_theta_at_given_p_soil_camp(psi_target=-100,
                                                                      theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_2'],
                                                                      psie=soil_parameters_dict['psie']['layer_2'],
                                                                      b_camp=soil_parameters_dict['b_camp']['layer_2'],),          
                          
                          'layer_3': compute_theta_at_given_p_soil_camp(psi_target=-100,
                                                                        theta_sat=soil_parameters_dict['saturation_capacity_campbell']['layer_3'],
                                                                        psie=soil_parameters_dict['psie']['layer_3'],
                                                                        b_camp=soil_parameters_dict['b_camp']['layer_3'],)})
                
                # Water values v_parameters -------------------------------------
    
                # Add v_field_capacity
                soil_parameters_dict['v_field_capacity'] = collections.defaultdict(
                    list,{'layer_1': convert_f_to_v(x=soil_parameters_dict['field_capacity']['layer_1'],
                                                    rfc=soil_parameters_dict['rfc']['layer_1'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                
                          'layer_2': convert_f_to_v(x=soil_parameters_dict['field_capacity']['layer_2'],
                                                    rfc=soil_parameters_dict['rfc']['layer_2'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                         
                          'layer_3': convert_f_to_v(x=soil_parameters_dict['field_capacity']['layer_3'],
                                                    rfc=soil_parameters_dict['rfc']['layer_3'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],)
                            })
                
                # Add v_saturation_capacity_campbellsoil_params_from_args['n_vg']
                soil_parameters_dict['v_saturation_capacity_campbell'] = collections.defaultdict(
                    list,{'layer_1':  convert_f_to_v(x=soil_parameters_dict['saturation_capacity_campbell']['layer_1'],
                                                     rfc=soil_parameters_dict['rfc']['layer_1'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                          
                          'layer_2':  convert_f_to_v(x=soil_parameters_dict['saturation_capacity_campbell']['layer_2'],
                                                     rfc=soil_parameters_dict['rfc']['layer_2'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                          
                          'layer_3':  convert_f_to_v(x=soil_parameters_dict['saturation_capacity_campbell']['layer_3'],
                                                     rfc=soil_parameters_dict['rfc']['layer_3'],
                                                     layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],)})
    
                # Add v_residual_capacity_campbell
                
                soil_parameters_dict['v_residual_capacity_campbell'] = collections.defaultdict(
                    list,{'layer_1': convert_f_to_v(x=soil_parameters_dict['residual_capacity_campbell']['layer_1'],
                                                    rfc=soil_parameters_dict['rfc']['layer_1'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                          
                          'layer_2': convert_f_to_v(x=soil_parameters_dict['residual_capacity_campbell']['layer_2'],
                                                    rfc=soil_parameters_dict['rfc']['layer_2'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],), 
                          
                          'layer_3': convert_f_to_v(x=soil_parameters_dict['residual_capacity_campbell']['layer_3'],
                                                    rfc=soil_parameters_dict['rfc']['layer_3'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],)})
    
                # Add v_wilting_point
                soil_parameters_dict['v_wilting_point'] = collections.defaultdict(
                    list,{'layer_1': convert_f_to_v(x=soil_parameters_dict['wilting_point']['layer_1'],
                                                    rfc=soil_parameters_dict['rfc']['layer_1'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_1'],),
                          
                          'layer_2': convert_f_to_v(x=soil_parameters_dict['wilting_point']['layer_2'],
                                                    rfc=soil_parameters_dict['rfc']['layer_2'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_2'],),
                          
                          'layer_3': convert_f_to_v(x=soil_parameters_dict['wilting_point']['layer_3'],
                                                    rfc=soil_parameters_dict['rfc']['layer_3'],
                                                    layer_thickness=soil_parameters_dict['soil_layers_thickness']['layer_3'],)})
    
                # Duplicate v_saturation_capacity, Don't know why
                #soil_parameters_dict['v_saturation_capacity'] = soil_parameters_dict['v_saturation_capacity_campbell']
    
                # For diagnostic (TAW)
                #soil_parameters_dict['v_soil_storage_capacity_wilt_campbell'] = np.sum(soil_parameters_dict['v_field_capacity']) - np.sum(soil_parameters_dict['v_wilting_point'])
    
                #soil_parameters_dict['v_soil_storage_capacity_res_campbell'] = np.sum(soil_parameters_dict['v_field_capacity']) - np.sum(soil_parameters_dict['v_residual_capacity_campbell'])
    
                #soil_parameters_dict['v_soil_storage_capacity'] = soil_parameters_dict['v_soil_storage_capacity_wilt_campbell']
    
                #print(
                #    f'Available water capacity Wilting: {soil_parameters_dict["v_soil_storage_capacity_wilt_campbell"]} mm'
                #)
                
                #print(
                #    f'Available water capacity Residual: {soil_parameters_dict["v_soil_storage_capacity_res_campbell"]} mm'
                #)
    
                #print('Can soil_params["v_soil_storage_capacity"] be negative?? Ask')
    
    else:
                # f'Option {soil_parameters_dict["pedo_transfer_formulation"]} not recognized.'
                raise ValueError("Failed to create soil parameters from input file")
    
    return soil_parameters_dict

#### __Example: Create soil parameters__


In [ ]:
from pysureau.pysureau_init import pysureau_init

In [ ]:
#pysureau_init('/tmp')


CSV parameter file saved at /tmp/pysureau_project_he3V99rs/1_parameter_files/soil_parameters_vg.csv
CSV parameter file saved at /tmp/pysureau_project_he3V99rs/1_parameter_files/soil_parameters_campbell.csv
CSV parameter file saved at /tmp/pysureau_project_he3V99rs/1_parameter_files/modeling_options.csv
CSV parameter file saved at /tmp/pysureau_project_he3V99rs/1_parameter_files/vegetation_parameters.csv
pysureau project created at /tmp/pysureau_project_he3V99rs


In [ ]:
prepare_soil_parameters(file_path='/tmp/pysureau_project_he3V99rs/1_parameter_files/soil_parameters_campbell.csv',)

3.0
7.0
11.0


defaultdict(list,
            {'b_camp': defaultdict(list,
                         {'layer_1': 1.0, 'layer_2': 1.0, 'layer_3': 1.0}),
             'g_soil_0': 2.0,
             'ksat_campbell': 3.0,
             'offset_psoil': 4.0,
             'pedo_transfer_formulation': 'campbell',
             'psie': defaultdict(list,
                         {'layer_1': -5.0, 'layer_2': -5.0, 'layer_3': -5.0}),
             'psie_camp': -0.025,
             'psoil_at_field_capacity': 6.0,
             'psoil_at_wilting_point': 1.5,
             'residual_capacity_campbell': defaultdict(list,
                         {'layer_1': 200.0,
                          'layer_2': 200.0,
                          'layer_3': 200.0}),
             'reset_swc': True,
             'saturation_capacity_campbell': defaultdict(list,
                         {'layer_1': 10.0, 'layer_2': 10.0, 'layer_3': 10.0}),
             'soil_evap': True,
             'soil_formulation': 'campbell',
             'water_soil_